### Data set contains train set and test set with two classes cars and planes.Predicting class using car or plane

## Unzipping the data

In [5]:
#!unzip v_data.zip

## Imported all needed packages

In [25]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import load_img
from sklearn import svm
import tensorflow as tf


## Reading images and appending to arrays

In [7]:
Categories=['cars','planes']
flat_data_arr=[] #input array
target_arr=[] #output array
datadir='v_data/train' 
for i in Categories:
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(224,224,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))


## Coverting to numpy arrays and intializing X and Y

In [8]:
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data)
df['Target']=target
x=df.iloc[:,:-1] 
y=df.iloc[:,-1] 
print(x.head(5))
print(y.head(5))

     0         1         2       ...    150525    150526    150527
0  0.133333  0.141176  0.129412  ...  0.258824  0.258824  0.258824
1  1.000000  1.000000  1.000000  ...  1.000000  1.000000  1.000000
2  0.992157  0.992157  0.984314  ...  0.996078  0.992157  1.000000
3  0.517647  0.654902  0.874510  ...  0.470588  0.435294  0.415686
4  0.686275  0.698039  0.670588  ...  0.815686  0.815686  0.784314

[5 rows x 150528 columns]
0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: int64


## Intializing variables for CNN

In [9]:
train_data_dir = 'v_data/train'
validation_data_dir = 'v_data/test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 10
batch_size = 16
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)

## Creating a model and adding layers to the model

In [26]:
model = Sequential()
model.call = tf.function(model.call)
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [27]:
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])

## Genetraing the data from the directories and fitting to the cnn model

In [29]:
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

model.fit_generator(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)


Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
25/25 [==============================] - 25s 930ms/step - loss: 1.0072 - accuracy: 0.6150 - val_loss: 0.4612 - val_accuracy: 0.9062
Epoch 2/10
25/25 [==============================] - 24s 936ms/step - loss: 0.5632 - accuracy: 0.7175 - val_loss: 0.4174 - val_accuracy: 0.8021
Epoch 3/10
25/25 [==============================] - 23s 925ms/step - loss: 0.4833 - accuracy: 0.7850 - val_loss: 0.2866 - val_accuracy: 0.8854
Epoch 4/10
25/25 [==============================] - 23s 912ms/step - loss: 0.4608 - accuracy: 0.7950 - val_loss: 0.3000 - val_accuracy: 0.8646
Epoch 5/10
25/25 [==============================] - 23s 922ms/step - loss: 0.3745 - accuracy: 0.8525 - val_loss: 0.3076 - val_accuracy: 0.8542
Epoch 6/10
25/25 [==============================] - 23s 912ms/step - loss: 0.3804 - accuracy: 0.8550 - val_loss: 0.2633 - val_accuracy: 0.9062
Epoch 7/10
25/25 [==============================] - 23s 915ms/step - loss: 0.3438 - accuracy: 0.8575 - val_loss: 0.2503 - val_accuracy: 0.8958

## Prediction of class using CNN model

In [30]:
model.save('my_model.h5')

In [23]:
from keras.preprocessing import image
img= load_img('v_data/test/cars/5.jpg', target_size=(224, 224))
img=image.img_to_array(img)
img=np.expand_dims(img,axis=0)
label = model.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", np.argmax(label,axis=1))

Predicted Class (0 - Cars , 1- Planes):  [0]


In [37]:
number_to_class = ['Car','Plane']
index = np.argmax(label,axis=1)
type(index[0])
number_to_class[index[0]]

'Car'

## Creating Svm model

In [ ]:
svc=svm.SVC()
svc.fit(x,y)

## Prediction of class using SVM model

In [ ]:
img_array=imread('v_data/test/cars/5.jpg')
img_resized=resize(img_array,(224,224,3))
img=img_resized.flatten()
img1=np.array([img])
p=svc.predict(img1)
print("Predicted Class (0 - Cars , 1- Planes): ", p)